In [13]:
def setUP(db):
    firstRun = True
    for index, row in db.iterrows():
        key = row['OfV Description'] + str(row['SKU'])
        if firstRun:
            res = list(OrderedDict.fromkeys(db.Month))
            ez = row['Returnability'] + row['Primary Pack Type'] + row['Primary Pack Group'] + row['HMex Pack Group'] + row['Pack Size']
            data = [row['Year'], row['GZ Split TVN-VDM'], row['Line Extension'], row['Multipack'], ez, key]
            name = ['Year', 'GZ', 'Line Extension', 'Multipack', 'EZ', 'key']
            for value in res:
                data.append(0)
                name.append(value)
            df = pd.DataFrame(data, name).T
            table = df
            firstRun = False
        try:
            month = row['Month']
            val = table['key'][table['key'] == key].index[0]
            table.loc[table.index[val], month] = row['Value']
        except:
            ez = row['Returnability'] + row['Primary Pack Type'] + row['Primary Pack Group'] + row['HMex Pack Group'] + row['Pack Size']
            data = [row['Year'], row['GZ Split TVN-VDM'], row['Line Extension'], row['Multipack'], ez, key]
            name = ['Year', 'GZ', 'Line Extension', 'Multipack', 'EZ', 'key']
            for value in res:
                data.append(0)
                name.append(value)
            df = pd.DataFrame(data, name).T
            table = pd.concat([table, df], axis = 0, ignore_index=True)
            month = row['Month']
            val = table['key'][table['key'] == key].index[0]
            table.loc[table.index[val], month] = row['Value']
    return table

In [14]:
def printTable(db, canalInt, canal, region, gz1, fabricante, marca_hom, ext_marca_hom,  diamante_precio_est, tipo_liq_est, liq, presentacion ,multipack , canalMA, marca_hom2, ext_marca_hom2, diamante_precio_hom, returnability, packtypeHOM, primaryPackGroup, groupPacksize, packSizeHOM, packtypeOk2, categorias, packSize1Home, pConcatenado1, pConcatenado2, manu, priceTierCMI, categoryCMI, alcoholSegmentsCMI, liquidSegmentsCMI, brandCondition, topLineSOg, secondarySOg):
    data = [db.Year, canalInt, canal, region, gz1, fabricante, marca_hom, ext_marca_hom,  diamante_precio_est, tipo_liq_est, liq, presentacion, multipack, canalMA, marca_hom2, ext_marca_hom2, diamante_precio_hom, returnability, packtypeHOM, primaryPackGroup, groupPacksize, packSizeHOM, packtypeOk2, categorias, packSize1Home, pConcatenado1, pConcatenado2, manu, priceTierCMI, categoryCMI, alcoholSegmentsCMI, liquidSegmentsCMI, brandCondition, topLineSOg, secondarySOg]
    name = ['AÑO', 'CANAL INTEGRADO', 'CANAL', 'REGION', 'GZ', 'FABRICANTE', 'MARCA_EST', 'EXT_MARCA_EST', 'DIAMANTE_PRECIO_EST', 'TIPO_LIQ_EST', 'LIQ', 'PRESENTACION', 'UNIDADES_EMPAQUE', 'CANAL_MA', 'MARCA_HOM', 'EXT_MARCA_HOM', 'DIAMANTE_PRECIO_HOM', 'Retunability', 'Packtype_HOM', 'Primary_Packtype_Group', 'Group_Packsize', 'Packsize_HOM', 'PACKTYPE_OK_2', 'Categorias', 'Packsize1_HOME', 'Packsize_concatenado1', 'Packsize_concatenado2', 'Manufacturer_CMI', 'Price_Tier_CMI', 'Category_CMI', 'Alcohol_Segments_CMI', 'Liquid_Segment_CMI', 'BRAND_CONDITION', 'Primary_Source_of_Growth', 'Secondary_Source_of_Growth']
    g = db.iloc[:,6:]
    monthlyValues = g.columns.tolist()
    for month in monthlyValues:
        data.append(db[month])
        name.append(month)
    df = pd.DataFrame(data, name).T
    df.to_excel("SIX.xlsx", index = False)

In [15]:
def generateDatabase():
    
    ## Lectura de documentos y datos
    try: 
        ## Leer los archivos
        sellOutRegion = openDataFile('Sell Out Region Six')
        sellOut = openDataFile('Sell Out Six')
        db = openDataFile('Input Six (Tabla del main DB con info de cervezas SIX)')
        ## Inicializar valores
        global setActual
        global lineaActual
        setActual = ''
        lineaActual = 10000
        flag = False
        gzCat = pd.read_excel('data/CatalogoSIX.xlsx', sheet_name = 'Gz')
    except: 
        print('ERROR')
        print('Se ha generado un problema en la lectura del archivo')
        print('Favor de revisar estos documentos y volver a intentar')

    ## Validacion de documentos y posibles errores
    try:
        result = validarDatos(db, sellOut, sellOutRegion)
        if result == 1: 
            print('Los documentos parecen estar en orden')
        elif result == 2:
            v1, v2, v3 = fixDB2(db['GZ Split TVN-VDM'].value_counts().index, sellOutRegion['Unnamed: 1'].value_counts().index, gzCat)
            print('El archivo tiene ' + v1 + ' posible linea(s) faltantes')
            print('Posibles errores: ' + ' | '.join(v2))
            print('Posibles valores faltantes: ' + ' | '.join(v3))
            flag = True
        elif result == 3:
            print('Sell Out')
            v1, v2, v3 = fixDB1(db['GZ Split TVN-VDM'].value_counts().index, sellOut['GZ'].value_counts().index, gzCat)
            print('El archivo tiene ' + v1 + ' posible linea(s) faltantes')
            print('Posibles errores: ' + ' | '.join(v2))
            print('Posibles valores faltantes: ' + ' | '.join(v3))
            flag = True
        elif result == 4 : 
            print('Sell Out')
            v1, v2, v3 = fixDB1(db['GZ Split TVN-VDM'].value_counts().index, sellOut['GZ'].value_counts().index, gzCat)
            print('El archivo tiene ' + v1 + ' posible linea(s) faltantes')
            print('Posibles errores: ' + ' | '.join(v2))
            print('Posibles valores faltantes: ' + ' | '.join(v3))
            print('Sell Out Region')
            v1, v2, v3 = fixDB2(db['GZ Split TVN-VDM'].value_counts().index, sellOutRegion['Unnamed: 1'].value_counts().index, gzCat)
            print('El archivo tiene ' + v1 + ' posible linea(s) faltantes')
            print('Posibles errores: ' + ' | '.join(v2))
            print('Posibles valores faltantes: ' + ' | '.join(v3))
    except: 
        print('ERROR')
        print('Se ha generado un problema en la validación del archivo')
        print('Favor de revisar estos documentos y volver a intentar')
        
    try: 
        db = setUP(db)
    except:
        return 1
    
    os.system('cls')
    flag = True
    while flag:
        os.system('cls')
        try:    ## Obtener los valores con la llave de canal                
            cn, cain, cama, fa  = setCanal(len(db)) 
            canal = pd.Series(cn)
            canalInt = pd.Series(cain)
            canalMA = pd.Series(cama)
            fabricante = pd.Series(fa)
            flag = False
        except:
            flag = errorMessage(setActual, lineaActual, valor)
        
    flag = True
    while flag: 
        gzCat = pd.read_excel('data/CatalogoSIX.xlsx', sheet_name = 'Gz')
        os.system('cls')   
        try: ## Obtener todos los valores con la llave de GZ
            re, g1 = setGZ(db['GZ'], gzCat)
            region = pd.Series(re)
            gz1 = pd.Series(g1)
            flag = False
        except:
            flag = errorMessage(setActual, lineaActual, valor)

            
    flag = True
    while flag:
        marcaCat = pd.read_excel('data/CatalogoSIX.xlsx', sheet_name = 'Marca')
        os.system('cls')
        try: ## Obtener todos los valores con la llave de Line Extension (Marca)
            mh, emh, dpe, tlq, li, mh2, emh2, dph, man, ptCMI, cCMI, asCMI, lsCMI, bc, tp, sd = setMarca(db['Line Extension'], marcaCat)
            marca_hom = pd.Series(mh)
            ext_marca_hom = pd.Series(emh)
            diamante_precio_est = pd.Series(dpe)
            tipo_liq_est = pd.Series(tlq)
            liq = pd.Series(li)
            marca_hom2 = pd.Series(mh2)
            ext_marca_hom2 = pd.Series(emh2)
            diamante_precio_hom = pd.Series(dph)
            manu = pd.Series(man)
            priceTierCMI = pd.Series(ptCMI)
            categoryCMI = pd.Series(cCMI)
            alcoholSegmentsCMI = pd.Series(asCMI)
            liquidSegmentsCMI = pd.Series(lsCMI)
            brandCondition = pd.Series(bc)
            topLineSOg = pd.Series(tp)
            secondarySOg = pd.Series(sd)
            flag = False
        except:
            flag = errorMessage(setActual, lineaActual, valor)
                   
    flag = True
    while flag:
        returnabilityCat = pd.read_excel('data/CatalogoSIX.xlsx', sheet_name = 'Returnability')
        os.system('cls')
        try: ## Obtener todos los valores con la llave de Returnability
            ret, prPT, prPG, hMex, ps, ptHOM, prPTG, gPS, psHOM, ptOK2, cat, ps1H, pc1, pc2, pr = setReturnability(db.EZ, returnabilityCat)
            returnability  = pd.Series(ret)
            primaryPackType = pd.Series(prPT)
            primaryPackGroup = pd.Series(prPG)
            hMexPack = pd.Series(hMex)
            packSize = pd.Series(ps)
            packtypeHOM = pd.Series(ptHOM)
            primaryPacktypeGroup = pd.Series(prPTG)
            groupPacksize = pd.Series(gPS)
            packSizeHOM = pd.Series(psHOM)
            packtypeOk2 = pd.Series(ptOK2)
            categorias = pd.Series(cat)
            packSize1Home = pd.Series(ps1H)
            pConcatenado1 = pd.Series(pc1)
            pConcatenado2 = pd.Series(pc2)
            presentacion = pd.Series(pr)
            flag = False
        except: 
            flag = errorMessage(setActual, lineaActual, valor)
            
    flag = True
    while flag:
        os.system('cls')
        try:
            mp = setUE(db.Multipack)
            multipack = pd.Series(mp)
            flag = False
        except:
            flag = errorMessage(setActual, lineaActual, valor)
            
    try:## Formatear todos los datos y crear un excel con ellos
        printTable(db, canalInt, canal, region, gz1, fabricante, marca_hom, ext_marca_hom,  diamante_precio_est, tipo_liq_est, liq, presentacion ,multipack , canalMA, marca_hom2, ext_marca_hom2, diamante_precio_hom, returnability, packtypeHOM, primaryPackGroup, groupPacksize, packSizeHOM, packtypeOk2, categorias, packSize1Home, pConcatenado1, pConcatenado2, manu, priceTierCMI, categoryCMI, alcoholSegmentsCMI, liquidSegmentsCMI, brandCondition, topLineSOg, secondarySOg)
        print('El documento se ha creado con exito!')
    except:
        print('ERROR')
        print('Se ha detectado un error en la creación del archivo, favor de volver a intentar')
        